In this lab, I will use Beautifulsoup 4 to scrap the data from a givien website. Then I will convert addresses into their equivalent latitude and longitude values. using the Foursquare API to explore neighborhoods in Toronto city. I will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the k-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

In [1]:
# import needed library

import pandas as pd 
from pandas.io.json import json_normalize
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium


from bs4 import BeautifulSoup # beautiful soup for scraping data. 

In [59]:
# using  bs4 for scraping the data from givien website.
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" 
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
#print(soup.prettify())

For this task, I have spent time to inspect to figure out the stucture of the webpage.

In [60]:
# Find table in that website
table = soup.find("table", {'class':'wikitable sortable'})
#table

In [4]:
# Scrap the table columns' name
title = table.find_all('th')
col = [i.text.strip() for i in title]
col

['Postal Code', 'Borough', 'Neighborhood']

In [5]:
# Getting rows of the table
row = table.find_all('tr')
len(row)

181

In [6]:
#loop through  table rows to get datas 
l = []
for tr in row:
    td = tr.find_all('td')
    row_data= [tr.text.strip() for tr in td]
    l.append(row_data)
data = l[1:]
data 
    

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M

In [7]:
# Create  new dataframe
toronto = pd.DataFrame(data,columns=col)
toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Data cleaning.

At we can see, the dataframe above contains some N/A values as "Not assigned". Will process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

To do so, first, I need to replace string 'Not assigned' with NaN values.

In [8]:
toronto = toronto.replace({'Not assigned':np.nan})
toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
# This step is to drop all rows that has values NaN in "Brough".
toronto = toronto[toronto['Borough'].notna()]
toronto.head()
        

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
toronto.shape

(103, 3)

# Using geocoder to assign lattitude and longitude 

At this steps, after doing some resreach on the internet, I have used <code>geocoder.arcgis</code> to get lattitude and longitude  of given Postal Code.

In [11]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
code = 'M3A'

while(lat_lng_coords is None):
  g = geocoder.arcgis('{},Toronto, Ontario'.format(code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(lat_lng_coords)

[43.75293455500008, -79.33564142299997]


In [12]:
#Loop  through the Postal Code columns to get the lattitude and longitude
pos= toronto['Postal Code'].values
pos
lat_lng_coords = None
latitude=[]
longitude =[]
for i in pos:
    g = geocoder.arcgis('{},Toronto, Ontario'.format(i))
    lat_lng_coords= g.latlng
    #print(lat_lng_coords)
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    
            

In [13]:
# Create 2 new columns lattitude and longitude for the dataframe
toronto["Latitude"]=latitude
toronto["Longitude"]=longitude
toronto= toronto.reset_index(drop=True)

In [61]:
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


##### Use geopy library to get the latitude and longitude values of Toronto city.

In [15]:
address = "Toronto, Ontario"
geolocator = Nominatim(user_agent="ny_explorer")
locat = geolocator.geocode(address)

lat = locat.latitude
lon = locat.longitude
print(lat,lon)

43.6534817 -79.3839347


#### Create a map of Toronto with neighborhoods superimposed on top.

In [16]:
toronto_map = folium.Map(location=[lat,lon],zoom_start=10)

for la,lo,neig,aera in zip(toronto['Latitude'],toronto['Longitude'],
                           toronto['Neighborhood'],toronto['Borough']):
    label = "{} {}".format(neig,aera)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [la,lo], radius=5,color='blue', popup= label, fill= True, fill_cofill_color = "#3186cc",
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)
toronto_map

# Explore Neighborhoods in Toronto

####  Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = '5HUS4HI54VP0KE1YGX1HFXZXSE0WH4OZSI1T4GII4NDPGYEE' # your Foursquare ID
CLIENT_SECRET = 'K4H2ZHSRVKGZRMJTF4NECQ04LKXVCV1OJHNARUDHBRTZKKKV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5HUS4HI54VP0KE1YGX1HFXZXSE0WH4OZSI1T4GII4NDPGYEE
CLIENT_SECRET:K4H2ZHSRVKGZRMJTF4NECQ04LKXVCV1OJHNARUDHBRTZKKKV


#### Create function to get venues categories

In [18]:
def get_categories_type(row):
    try:
        category_list = row['catogories']
    except:
        category_list = row['venue.categories']
    if len(category_list) ==0:
        return None
    else:
        return category_list[0]['name']

#### Create a function that will make request API and get all the venues info that we need for later analyst. 

In [19]:

def discover_toronto(names,latitude,longitude, radius=500, limit = 100):
    venues_list=[]
    for name, lat, long in zip(names,latitude,longitude):
        print(name)
        # API request
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius,limit)
        # get request
        result = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(name,lat,long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in result])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [20]:
# Call the function to create dataframe
toronto_neiborh = discover_toronto(toronto["Neighborhood"],toronto["Latitude"],
                                   toronto["Longitude"])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [21]:
# New dataframe
toronto_neiborh.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752935,-79.335641,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.728102,-79.311890,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.728102,-79.311890,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [22]:
toronto_neiborh.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,2,2,2,2,2,2
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,64,64,64,64,64,64
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",43,43,43,43,43,43
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100


In [23]:
print(" there  are {}  unique venue around".format(len(toronto_neiborh['Venue Category'].unique())))

 there  are 268  unique venue around


# Analyze Each Neighborhood

In [24]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [25]:
# one hot encoding
dummie_data  = pd.get_dummies(toronto_neiborh[["Venue Category"]],prefix="", prefix_sep="")


dummie_data.head(3)

,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fireworks Store,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [26]:
# Add back "Neighborhood" to the dataframe
dummie_data["Neighborhood"]= toronto_neiborh["Neighborhood"]
dummie_data.head(5)


,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fireworks Store,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [27]:
dummie_data.shape

(2270, 268)

In [28]:
# move Neiborhood to the first column.
ne = dummie_data.pop("Neighborhood")
dummie_data.insert(0,"Neighborhood",ne)
dummie_data.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fireworks Store,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [29]:
toronto_group = dummie_data.groupby("Neighborhood").mean().reset_index()
toronto_group.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fireworks Store,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

#### Let's print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in toronto_group['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_group[toronto_group['Neighborhood'] == hood].T.reset_index()
    #print(temp)
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
              venue  freq
0       Supermarket   0.2
1      Skating Rink   0.2
2    Breakfast Spot   0.2
3  Sushi Restaurant   0.2
4   Badminton Court   0.2


----Alderwood, Long Branch----
            venue  freq
0    Dance Studio   0.1
1     Pizza Place   0.1
2     Gas Station   0.1
3  Sandwich Place   0.1
4     Coffee Shop   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0                 Bank  0.10
1          Coffee Shop  0.10
2  Fried Chicken Joint  0.05
3     Sushi Restaurant  0.05
4          Supermarket  0.05


----Bayview Village----
                        venue  freq
0  Construction & Landscaping   0.5
1                       Trail   0.5
2           Accessories Store   0.0
3                      Museum   0.0
4                Noodle House   0.0


----Bedford Park, Lawrence Manor East----
                     venue  freq
0           Sandwich Place  0.09
1              Coffee Shop  0.09
2       Italian Restaurant  0.09

                     venue  freq
0               Playground   0.5
1                     Park   0.5
2        Accessories Store   0.0
3      Moroccan Restaurant   0.0
4  New American Restaurant   0.0


----Humewood-Cedarvale----
           venue  freq
0  Grocery Store  0.17
1   Hockey Arena  0.17
2          Field  0.17
3          Trail  0.17
4   Tennis Court  0.17


----India Bazaar, The Beaches West----
                  venue  freq
0  Fast Food Restaurant  0.10
1        Sandwich Place  0.10
2        Ice Cream Shop  0.05
3            Restaurant  0.05
4            Steakhouse  0.05


----Islington Avenue, Humber Valley Village----
               venue  freq
0               Park  0.50
1       Skating Rink  0.25
2     Baseball Field  0.25
3  Accessories Store  0.00
4             Museum  0.00


----Kennedy Park, Ionview, East Birchmount Park----
               venue  freq
0  Convenience Store  0.25
1   Department Store  0.25
2        Coffee Shop  0.25
3     Discount Store  0.25
4  Accessorie

               venue  freq
0     Sandwich Place  0.09
1               Café  0.09
2  Indian Restaurant  0.04
3       Liquor Store  0.04
4        Pizza Place  0.04


----The Beaches----
               venue  freq
0                Pub  0.25
1  Health Food Store  0.25
2              Trail  0.25
3  Accessories Store  0.00
4      Movie Theater  0.00


----The Danforth West, Riverdale----
              venue  freq
0     Grocery Store   0.2
1          Bus Line   0.2
2              Park   0.2
3  Business Service   0.2
4    Discount Store   0.2


----The Kingsway, Montgomery Road, Old Mill North----
               venue  freq
0               Park   0.5
1             Lounge   0.5
2  Accessories Store   0.0
3             Museum   0.0
4       Night Market   0.0


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.17
1        Yoga Studio  0.08
2        Supermarket  0.08
3       Intersection  0.08
4      Grocery Store  0.08


----Toronto Dominion Centre, Design Exchange----
 

In [31]:
toronto_group.iloc[1,:]

Neighborhood                                Alderwood, Long Branch
Accessories Store                                                0
Afghan Restaurant                                                0
Airport                                                          0
American Restaurant                                              0
Art Gallery                                                      0
Art Museum                                                       0
Arts & Crafts Store                                              0
Asian Restaurant                                                 0
Athletics & Sports                                             0.1
Auto Dealership                                                  0
Auto Garage                                                      0
BBQ Joint                                                        0
Baby Store                                                       0
Badminton Court                                               

In [32]:
# Create a funtions that return most common venues
def most_common_venues(row,num):
    row_category = row.iloc[1:]
    row_categoryo_sorted = row_category.sort_values(ascending=False)
    
    return row_categoryo_sorted.index.values[0:num]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [33]:
num_top = 10

indicator = ["st","nd","rd"]
colunms = ["Neiborhood"]

# Format the columns' name 
for ind in np.arange(num_top):
    try:
        colunms.append("{}{} Most venues".format(ind+1,indicator[ind]))
    except:
        colunms.append("{}th most venues".format(ind+1))

top10_data  = pd.DataFrame(columns=colunms)
top10_data['Neiborhood'] = toronto_group['Neighborhood']

# Calling the funtion most_common_venues  
for i in np.arange(toronto_group.shape[0]):
    top10_data.iloc[i,1:] = most_common_venues(toronto_group.iloc[i,:],num_top)
top10_data.head()

,Neiborhood,1st Most venues,2nd Most venues,3rd Most venues,4th most venues,5th most venues,6th most venues,7th most venues,8th most venues,9th most venues,10th most venues
0,Agincourt,Sushi Restaurant,Skating Rink,Breakfast Spot,Badminton Court,Supermarket,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
1,"Alderwood, Long Branch",Convenience Store,Pub,Gas Station,Coffee Shop,Gym,Dance Studio,Athletics & Sports,Sandwich Place,Pharmacy,Pizza Place
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Middle Eastern Restaurant,Bridal Shop,Shopping Mall,Mobile Phone Shop,Supermarket,Fried Chicken Joint,Sushi Restaurant
3,Bayview Village,Construction & Landscaping,Trail,Field,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Eastern European Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Indian Restaurant,Juice Bar,Liquor Store,Japanese Restaurant,Thai Restaurant,Pub,Sports Club


# Cluster Neighborhoods

In [62]:
# import  KMeans library. 
from sklearn.cluster import KMeans

#drop sting column 
toronto_group_cluster = toronto_group.drop('Neighborhood',1)
toronto_group_cluster.head()

,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fireworks Store,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

#### Build the model

In [63]:
# run k-means clustering
kmean= KMeans(init="k-means++", n_clusters=3, n_init=12).fit(toronto_group_cluster)

# check cluster labels generated for each row in the dataframe
label= kmean.labels_
label[0:10]

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 2], dtype=int32)

In [64]:
# add clustering labels to the dataframe
top10_data["Cluster"]=label
top10_data.head()

,Neiborhood,1st Most venues,2nd Most venues,3rd Most venues,4th most venues,5th most venues,6th most venues,7th most venues,8th most venues,9th most venues,10th most venues,Cluster
0,Agincourt,Sushi Restaurant,Skating Rink,Breakfast Spot,Badminton Court,Supermarket,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,2
1,"Alderwood, Long Branch",Convenience Store,Pub,Gas Station,Coffee Shop,Gym,Dance Studio,Athletics & Sports,Sandwich Place,Pharmacy,Pizza Place,2
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Middle Eastern Restaurant,Bridal Shop,Shopping Mall,Mobile Phone Shop,Supermarket,Fried Chicken Joint,Sushi Restaurant,2
3,Bayview Village,Construction & Landscaping,Trail,Field,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Eastern European Restaurant,1
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Indian Restaurant,Juice Bar,Liquor Store,Japanese Restaurant,Thai Restaurant,Pub,Sports Club,2


   #### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood. by merging Toronto dataframe with top10_data. 

In [65]:
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


In [66]:
toronto_final = toronto
toronto_final = toronto_final.join(top10_data.set_index("Neiborhood"),on="Neighborhood")
toronto_final.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st Most venues,2nd Most venues,3rd Most venues,4th most venues,5th most venues,6th most venues,7th most venues,8th most venues,9th most venues,10th most venues,Cluster
0,M3A,North York,Parkwoods,43.752935,-79.335641,Park,Food & Drink Shop,Fireworks Store,Yoga Studio,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Field,0.0
1,M4A,North York,Victoria Village,43.728102,-79.311890,Coffee Shop,Pizza Place,Portuguese Restaurant,French Restaurant,Park,Intersection,Yoga Studio,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,2.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,Pub,Café,Athletics & Sports,Coffee Shop,Food Truck,Mediterranean Restaurant,Performing Arts Venue,Mexican Restaurant,French Restaurant,Seafood Restaurant,2.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,Clothing Store,Furniture / Home Store,Women's Store,Bookstore,Food Court,Men's Store,Restaurant,Toy / Game Store,American Restaurant,Cosmetics Shop,2.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,College Auditorium,Bookstore,Burrito Place,Smoothie Shop,Sandwich Place,Pharmacy,2.0
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667481,-79.528953,Park,Baseball Field,Skating Rink,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Yoga Studio,Eastern European Restaurant,0.0
6,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913,Trail,Yoga Studio,Field,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Eastern European Restaurant,1.0
7,M3B,North York,Don Mills,43.748900,-79.357220,Athletics & Sports,Bank,Burger Joint,Park,Other Great Outdoors,Restaurant,Trail,Coffee Shop,Farmers Market,Electronics Store,2.0
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529,Pizza Place,Fast Food Restaurant,Gym / Fitness Center,Pharmacy,Pet Store,Gastropub,Rock Climbing Spot,Breakfast Spot,Athletics & Sports,Bank,2.0
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Hotel,Café,Italian Restaurant,Bar,Cosmetics Shop,Theater,Sandwich Place,2.0


# Visualize the resulting clusters

At this step,  I have run into a problem that there was an NaN value in my Cluster. Therefore, it is giving error when trying to visualize the map.

In [67]:
#check dtype
toronto_final.dtypes

Postal Code          object
Borough              object
Neighborhood         object
Latitude            float64
Longitude           float64
1st Most venues      object
2nd Most venues      object
3rd Most venues      object
4th most venues      object
5th most venues      object
6th most venues      object
7th most venues      object
8th most venues      object
9th most venues      object
10th most venues     object
Cluster             float64
dtype: object

In [68]:
# Check if there is any Nan value on the Cluster
toronto_final["Cluster"].isnull().values.any()


True

In [69]:
#Find out where it is
toronto_final['Cluster'].values

array([ 0.,  2.,  2.,  2.,  2.,  0.,  1.,  2.,  2.,  2.,  2.,  2.,  0.,
        2.,  2.,  2.,  0.,  2.,  2.,  2.,  2.,  0.,  2.,  2.,  2.,  2.,
        1.,  0.,  2.,  2.,  2.,  0.,  2.,  2.,  2.,  2.,  0.,  2.,  2.,
        1.,  2.,  0.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  0.,  2.,  2.,
        2.,  2.,  2.,  2.,  1.,  0.,  2.,  2.,  2.,  2.,  2.,  2.,  0.,
        2.,  0.,  0.,  2.,  0.,  2.,  2.,  2.,  0.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  0.,  2.,  2.,  2.,  2.,  0.,  2.,  2.,
        0.,  2.,  2.,  2., nan,  2.,  2.,  0.,  2.,  2.,  0.,  2.])

In [70]:
# As we can noted, the nan value is located at the end of the dataframe. It's is on the line 95 with the
# postal code M1X.
toronto_final.tail(15)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st Most venues,2nd Most venues,3rd Most venues,4th most venues,5th most venues,6th most venues,7th most venues,8th most venues,9th most venues,10th most venues,Cluster
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.610075,-79.499751,Yoga Studio,Park,Skating Rink,Tennis Court,Grocery Store,Convenience Store,Dance Studio,Electronics Store,Flower Shop,Fish Market,0.0
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.744055,-79.581203,Pizza Place,Park,Caribbean Restaurant,Beer Store,Japanese Restaurant,Liquor Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Hardware Store,2.0
90,M1W,Scarborough,"Steeles West, L'Amoreaux West",43.800530,-79.321830,Chinese Restaurant,Gym,Gym Pool,Electronics Store,Burger Joint,Bank,Coffee Shop,Thrift / Vintage Store,Grocery Store,Sandwich Place,2.0
91,M4W,Downtown Toronto,Rosedale,43.681893,-79.376706,Playground,Grocery Store,Candy Store,Park,Fish & Chips Shop,Fireworks Store,Film Studio,Field,Fast Food Restaurant,Dumpling Restaurant,0.0
92,M5W,Downtown Toronto,Stn A PO Boxes,43.648700,-79.385450,Coffee Shop,Hotel,Japanese Restaurant,Restaurant,Café,Asian Restaurant,Sushi Restaurant,Bar,Mediterranean Restaurant,Sandwich Place,2.0
93,M8W,Etobicoke,"Alderwood, Long Branch",43.600895,-79.540387,Convenience Store,Pub,Gas Station,Coffee Shop,Gym,Dance Studio,Athletics & Sports,Sandwich Place,Pharmacy,Pizza Place,2.0
94,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.711615,-79.588351,Rental Car Location,Hotel,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fireworks Store,Film Studio,Dumpling Restaurant,Field,2.0
95,M1X,Scarborough,Upper Rouge,43.834768,-79.204101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667656,-79.367326,Pizza Place,Coffee Shop,Restaurant,Café,Italian Restaurant,Pub,Bakery,Park,Chinese Restaurant,Liquor Store,2.0
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648269,-79.381884,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,American Restaurant,Seafood Restaurant,Asian Restaurant,Steakhouse,2.0


In [71]:
# Drop nan value
toronto_final= toronto_final.dropna()

In [72]:
toronto_final['Cluster'].values

array([0., 2., 2., 2., 2., 0., 1., 2., 2., 2., 2., 2., 0., 2., 2., 2., 0.,
       2., 2., 2., 2., 0., 2., 2., 2., 2., 1., 0., 2., 2., 2., 0., 2., 2.,
       2., 2., 0., 2., 2., 1., 2., 0., 2., 2., 2., 2., 2., 2., 2., 0., 2.,
       2., 2., 2., 2., 2., 1., 0., 2., 2., 2., 2., 2., 2., 0., 2., 0., 0.,
       2., 0., 2., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 2.,
       2., 2., 2., 0., 2., 2., 0., 2., 2., 2., 2., 2., 0., 2., 2., 0., 2.])

In [82]:
# Convert Cluster to integer value.
toronto_final["Cluster"]=toronto_final["Cluster"].astype('int32')

In [83]:
final_map = folium.Map(location=[lat,lon],zoom_start=10)

# set color scheme for the clusters
kclusters=3
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


for la,lo,code,neig,cluster in zip(toronto_final['Latitude'],toronto_final['Longitude'],
                                   toronto_final['Postal Code'],toronto_final['Neighborhood'],
                                   toronto_final['Cluster']):
    label = "{}, Postal code {}, Cluster {}".format(aera,code,cluster)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([la,lo],radius=5,
                        popup= label,
                        #color=rainbow[cluster-1],
                        color=rainbow[cluster-1],
                        fill= True, 
                        fill_color = rainbow[cluster-1],
                        fill_opacity=0.7,
                        parse_html=False).add_to(final_map)
                        

In [84]:
final_map

In [76]:
cluster0 = toronto_final.loc[toronto_final['Cluster']==0].reset_index(drop=True)
cluster0

,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st Most venues,2nd Most venues,3rd Most venues,4th most venues,5th most venues,6th most venues,7th most venues,8th most venues,9th most venues,10th most venues,Cluster
0,M3A,North York,Parkwoods,43.752935,-79.335641,Park,Food & Drink Shop,Fireworks Store,Yoga Studio,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Field,0
1,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667481,-79.528953,Park,Baseball Field,Skating Rink,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Yoga Studio,Eastern European Restaurant,0
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368,Bar,Field,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Yoga Studio,Fountain,0
3,M6C,York,Humewood-Cedarvale,43.691730,-79.430013,Hockey Arena,Trail,Grocery Store,Field,Park,Tennis Court,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,0
4,M6E,York,Caledonia-Fairbanks,43.689118,-79.450650,Park,Sporting Goods Shop,Gym,Women's Store,Beer Store,Mexican Restaurant,Bakery,Falafel Restaurant,Farm,Farmers Market,0
5,M2H,North York,Hillcrest Village,43.802556,-79.356566,Dog Run,Residential Building (Apartment / Condo),Park,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Dumpling Restaurant,0
6,M6H,West Toronto,"Dufferin, Dovercourt Village",43.664910,-79.438664,Grocery Store,Park,Pharmacy,Bus Line,Art Gallery,Athletics & Sports,Smoke Shop,Bank,Café,Bakery,0
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.623750,-79.369200,Harbor / Marina,Park,Theme Park,Farm,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Yoga Studio,0
8,M4K,East Toronto,"The Danforth West, Riverdale",43.683424,-79.354564,Park,Grocery Store,Bus Line,Discount Store,Business Service,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,0
9,M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.713530,-79.488724,Bakery,Basketball Court,Trail,Park,Fast Food Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Field,0


In [77]:
cluster1= toronto_final.loc[toronto_final['Cluster']==1].reset_index(drop=True)
cluster1

,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st Most venues,2nd Most venues,3rd Most venues,4th most venues,5th most venues,6th most venues,7th most venues,8th most venues,9th most venues,10th most venues,Cluster
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913,Trail,Yoga Studio,Field,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Eastern European Restaurant,1
1,M1H,Scarborough,Cedarbrae,43.768791,-79.238813,Construction & Landscaping,Gaming Cafe,Eastern European Restaurant,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fireworks Store,Film Studio,Field,1
2,M2K,North York,Bayview Village,43.780607,-79.376921,Construction & Landscaping,Trail,Field,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Eastern European Restaurant,1
3,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.695075,-79.487179,Construction & Landscaping,Playground,Coffee Shop,Health & Beauty Service,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,1


In [78]:
cluster2=toronto_final.loc[toronto_final['Cluster']==2].reset_index(drop=True)
cluster2

,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st Most venues,2nd Most venues,3rd Most venues,4th most venues,5th most venues,6th most venues,7th most venues,8th most venues,9th most venues,10th most venues,Cluster
0,M4A,North York,Victoria Village,43.728102,-79.311890,Coffee Shop,Pizza Place,Portuguese Restaurant,French Restaurant,Park,Intersection,Yoga Studio,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,2
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,Pub,Café,Athletics & Sports,Coffee Shop,Food Truck,Mediterranean Restaurant,Performing Arts Venue,Mexican Restaurant,French Restaurant,Seafood Restaurant,2
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,Clothing Store,Furniture / Home Store,Women's Store,Bookstore,Food Court,Men's Store,Restaurant,Toy / Game Store,American Restaurant,Cosmetics Shop,2
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,College Auditorium,Bookstore,Burrito Place,Smoothie Shop,Sandwich Place,Pharmacy,2
4,M3B,North York,Don Mills,43.748900,-79.357220,Athletics & Sports,Bank,Burger Joint,Park,Other Great Outdoors,Restaurant,Trail,Coffee Shop,Farmers Market,Electronics Store,2
5,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529,Pizza Place,Fast Food Restaurant,Gym / Fitness Center,Pharmacy,Pet Store,Gastropub,Rock Climbing Spot,Breakfast Spot,Athletics & Sports,Bank,2
6,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Hotel,Café,Italian Restaurant,Bar,Cosmetics Shop,Theater,Sandwich Place,2
7,M6B,North York,Glencairn,43.707279,-79.447500,Pizza Place,Asian Restaurant,Fast Food Restaurant,Sushi Restaurant,Gas Station,Japanese Restaurant,Italian Restaurant,Grocery Store,Mediterranean Restaurant,Pub,2
8,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650023,-79.554089,Pizza Place,Chinese Restaurant,Tea Room,Sandwich Place,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,2
9,M3C,North York,Don Mills,43.722143,-79.352023,Athletics & Sports,Bank,Burger Joint,Park,Other Great Outdoors,Restaurant,Trail,Coffee Shop,Farmers Market,Electronics Store,2


We can name 3 cluster: 

Cluster0: Living area (with mostly park, playground, gym, school; and some small businesses)
Cluster1: Developing area( with most of them are Construction & Landscaping, farm)
Cluster2: Business area (with lots of business venues)